In [6]:
from pymodbus.client.sync import ModbusSerialClient as mbc
import keyboard
import csv
import time
import logging 
from datetime import datetime
import pandas as pd 
import pyvisa

In [2]:
client = mbc(method = "rtu", port="COM8",stopbits = 1, bytesize = 8, parity = 'N', baudrate= 1200)
#connect to client
client.connect()
while True:
    

True

In [3]:
ray1 = []
ray2 = []

input_reg = client.read_input_registers(200,count=2, unit=1)
#put data into 2 arrays
ray1.append(input_reg.registers[0])
ray2.append(input_reg.registers[1])

AttributeError: 'ModbusIOException' object has no attribute 'registers'

In [32]:
 client.close()

# Test

Code that will run and turn each string of LEDs on and will read the temp sensor on the board 

In [33]:
#address of register
address = 100
#value to be written to register 
value = 100
client = mbc(method = "rtu", port="COM8",stopbits = 1, bytesize = 8, parity = 'N', baudrate= 115200, retries=0, timeout=.1)
#connect to client
client.connect()
#.write_register(address, value)
ls = [2,2,50,50]
lb = [0,0,0,0]
try:
    for i in range(1,8):
        
        client.write_registers(int(address), ls, unit = i)
        time.sleep(1)
#         input_reg = client.read_input_registers(200,count=1, unit=i)
        time.sleep(1)
        client.write_registers(int(address), lb, unit = i)
        print(i)
        input_reg = client.read_input_registers(200,count=1, unit=1)
#         print(input_reg)
        print(input_reg.registers)
        time.sleep(.005)
except Exception as e:
    print(e)
    
print(input_reg.registers)
client.close()


1
[218]
2
[218]
3
[216]
4
[216]
5
[216]
6
[218]
7
[218]
[218]


In [13]:
client = mbc(method = "rtu", port="COM8",stopbits = 1, bytesize = 8, parity = 'N', baudrate= 115200, retries=0, timeout=.1)
#connect to client
client.connect()
client.write_coil(300, True)
client.close()

In [5]:
client.write_coil(300, True)

pymodbus.exceptions.ModbusIOException(pymodbus.exceptions.InvalidMessageReceivedException('No response received, expected at least 2 bytes (0 received)'),
                                      5)

In [9]:
input_reg = client.read_input_registers(200,count=1, unit=1)
print(input_reg)

ReadInputRegistersResponse (1)


In [40]:
client.close()

In [25]:
client = mbc(method = "rtu", port="COM8",stopbits = 1, bytesize = 8, parity = 'N', baudrate= 115200, retries=0, timeout=.1)
#connect to client
client.connect()
time.sleep(1)
try:
    for i in range(1,4):
        print(i)
        input_reg = client.read_input_registers(200,count=1, unit=1)
#         print(input_reg)
        print(input_reg.registers)
        time.sleep(.005)
except Exception as e:
    print(e)
time.sleep(1)
client.close()

1
[213]
2
[213]
3
[213]


# Next I will build the test program

1. Each chanel of the board will be lit up at low power
2. The temp data from each temp sensor will be recorded every time a new string is turned on


In [44]:


def Continutiy_Run():
    # adress of each register
    address = 100
    testName = input("Please input name of test:")
    #values to go to each register
    write = 2
    off = 0

    #array of values for tmp sensors
    tmpData_Array = [0,0,0,0,0,0,0]

    client = mbc(method = "rtu", port="COM8",stopbits = 1, bytesize = 8, parity = 'N', baudrate= 115200, retries=0, timeout=.1)
    
    dirLoc = "C:\\Users\\aaron\\Dropbox\\NewForge\\Summer2023\\Europa_Simulator\\" + testName + ".csv"
    header = ['Board', 'Temp1', 'Temp2', 'Temp3', 'Temp4', 'Temp5', 'Temp6', 'Temp7']
    df = pd.DataFrame(columns= header)
    
    #creats a new csv file at the dirLoc path
    try:
        with open(dirLoc, 'w') as path:
            #defines the csv writer to 
            writer = csv.writer(path)
            #writes the header for the file
            writer.writerow(header)
    except Excpetion as e:
        print(e)
        print('Failed to create file and write to it. Most likely file is still open, close it dummy')

    
    #connect to client
    client.connect()
    #.write_register(address, value)

    try:
        #first string on and off
        for i in range(1,8):
            print(i)
            for x in range(4):
                print(x)
                if(x >= 2):
                    write = 50
                client.write_registers((int(address) + x), write, unit = i)
                time.sleep(1)
                client.write_registers((int(address) + x), off, unit = i)
                
                for l in range(1,8):
                    input_reg = client.read_input_registers(200,count=1, unit=l)
                    tmpData_Array[l-1] = input_reg.registers

        
                df = df.append({'Board': i , 'Temp1': tmpData_Array[0][0] , 'Temp2': tmpData_Array[1][0], 'Temp3': tmpData_Array[2][0] , 'Temp4': tmpData_Array[3][0], 'Temp5': tmpData_Array[4][0], 'Temp6': tmpData_Array[5][0] , 'Temp7': tmpData_Array[6][0]}, ignore_index=True)

    except Exception as e:
        print(e)
    try:
        df.to_csv(dirLoc)
    except Excpetion as e:
        print(e)
        print('Failed to create file and write to it. Most likely file is still open, close it dummy')
    client.close()


In [45]:
Continutiy_Run()

Please input name of test:Test1
1
0
1
2
3
2
0
1
2
3
3
0
1
2
3
4
0
1
2
3
5
0
1
2
3
6
0
1
2
3
7
0
1
2
3


# Thermal Test

1. Ramp all LEDS to full power 
2. Once X temp is reached
3. Start recording temp data every second for 2 minutes

In [5]:
import time 

def Thermal_Test():
    client = mbc(method = "rtu", port="COM8",stopbits = 1, bytesize = 8, parity = 'N', baudrate= 115200, retries=0, timeout=.1)
    
    address = 100
    tempTo = input("Please input the temperature you would like to start the test at :")
    testName = input("Please input the name of the test:")
    la = [50,50,50,50]
    lb = [100,100,100,100]
    lc = [150,150,150,150]
    ld = [200,200,200,200]
    le = [255,255,255,255]
    
    off = [0,0,0,0]
    tmpData_Array = []
    
    dirLoc = "C:\\Users\\aaron\\Dropbox\\NewForge\\Summer2023\\Europa_Simulator\\" + str(testName) + ".csv"
    header = ['Time', 'Temp1', 'Temp2', 'Temp3', 'Temp4', 'Temp5', 'Temp6', 'Temp7']
    df = pd.DataFrame(columns= header)
    
    #first ramp up the LEDS
    client.connect()
    
    for i in range(1,8):
        client.write_registers(int(address), la, unit = i)
        time.sleep(.5)
    for i in range(1,8):       
        client.write_registers(int(address), lb, unit = i)
        time.sleep(.5)
    for i in range(1,8):  
        client.write_registers(int(address), lc, unit = i)
        time.sleep(.5)
    for i in range(1,8):
        client.write_registers(int(address), ld, unit = i)
        time.sleep(.5)
    for i in range(1,8):            
        client.write_registers(int(address), le, unit = i)
        time.sleep(.5)

    
    
    while(int(tempTo) *10 > int(temp)):
        time.sleep(.005)
        data = client.read_input_registers(200,count=1, unit=1)
        temp = data.registers
        
    #end time = time + 60 * num minutes
    t_end = time.time() + 60 * 2
    
    while time.time() < t_end:
        time.sleep(.5)
        for i in range(1,8):
            input_reg = client.read_input_registers(200,count=1, unit=i)
            tmpData_Array[i-1] = input_reg.registers
        df = df.append({'Time': time.time() , 'Temp1': tmpData_Array[0][0] , 'Temp2': tmpData_Array[1][0], 'Temp3': tmpData_Array[2][0] , 'Temp4': tmpData_Array[3][0], 'Temp5': tmpData_Array[4][0], 'Temp6': tmpData_Array[5][0] , 'Temp7': tmpData_Array[6][0]}, ignore_index=True)
        time.sleep(.005)
       
        
    #turn every pannel off
    for i in range(1,8):
        client.write_registers(int(address), off, unit = i)
        
    try:
        df.to_csv(dirLoc)
        client.close()
        return df
    except Excpetion as e:
        print(e)
        print('Failed to create file and write to it. Most likely file is still open, close it dummy')
    

In [2]:
#SINGLE TRIAL RUN#



import time 

def Thermal_Test_Test():
    address = 100
    tempTo = input("Please input the temperature you would like to start the test at :")
    client = mbc(method = "rtu", port="COM8",stopbits = 1, bytesize = 8, parity = 'N', baudrate= 115200, retries=0, timeout=.1)
    client.connect()
    
    #     testName = input("Please input the name of the test:")
    la = [50,50,2,2]
    lb = [100,100,4,4]
    lc = [150,150,150,150]
    ld = [200,200,200,200]
    le = [255,255,255,255]
    
    off = [0,0,0,0]
    tmpData_Array = []
    
#     dirLoc = "C:\\Users\\aaron\\Dropbox\\NewForge\\Summer2023\\Europa_Simulator\\" + str(testName) + ".csv"
    header = ['Time', 'Temp1']
    df = pd.DataFrame(columns= header)
    
    #first ramp up the LEDS
#     for i in range(1,8):
    client.write_registers(int(address), la, unit = 1)
    time.sleep(.5)
#     for i in range(1,8):       
    client.write_registers(int(address), lb, unit = 1)
    time.sleep(.5)
#     for i in range(1,8):  
#         client.write_registers(int(address), lc, unit = i)
#         time.sleep(.5)
#     for i in range(1,8):
#         client.write_registers(int(address), ld, unit = i)
#         time.sleep(.5)
#     for i in range(1,8):            
#         client.write_registers(int(address), le, unit = i)
#         time.sleep(.5)

    data = client.read_input_registers(200,count=1, unit=1)
    temp = data.registers
    
    while(int(tempTo) *10 > int(temp[0])):
        time.sleep(.005)
        data = client.read_input_registers(200,count=1, unit=1)
        temp = data.registers
    
    print(temp[0])
    #end time = time + 60 * num minutes
    t_end = time.time() + 60 * 2
    
    while time.time() < t_end:
        time.sleep(.5)
        input_reg = client.read_input_registers(200,count=1, unit=1)
        tmpData_Array = input_reg.registers
        df = df.append({'Time': time.time() , 'Temp1': tmpData_Array[0]}, ignore_index=True)
        time.sleep(.005)
       
        
    #turn every pannel off
#     for i in range(1,8):
    client.write_registers(int(address), off, unit = 1)
    client.close()
    
    return df
    
    
#     try:
# #         df.to_csv(dirLoc)
#     except Excpetion as e:
#         print(e)
#         print('Failed to create file and write to it. Most likely file is still open, close it dummy')
    

In [3]:
df = Thermal_Test_Test()

Please input the temperature you would like to start the test at :29
291


In [4]:
df

,Time,Temp1
0,1.686082e+09,294.0
1,1.686082e+09,296.0
2,1.686082e+09,308.0
3,1.686082e+09,311.0
4,1.686082e+09,318.0
5,1.686082e+09,321.0
6,1.686082e+09,326.0
7,1.686082e+09,340.0
8,1.686082e+09,335.0
9,1.686082e+09,345.0


In [95]:



address = 100
tempTo = input("Please input the temperature you would like to start the test at :")
#     testName = input("Please input the name of the test:")
la = [50,50,2,2]
lb = [100,100,4,4]
lc = [150,150,150,150]
ld = [200,200,200,200]
le = [255,255,255,255]

off = [0,0,0,0]
tmpData_Array = []

#     dirLoc = "C:\\Users\\aaron\\Dropbox\\NewForge\\Summer2023\\Europa_Simulator\\" + str(testName) + ".csv"
header = ['Time', 'Temp1']
df = pd.DataFrame(columns= header)

#first ramp up the LEDS
#     for i in range(1,8):
client.write_registers(int(address), la, unit = 1)
time.sleep(.5)
#     for i in range(1,8):       
client.write_registers(int(address), lb, unit = 1)
time.sleep(.5)
#     for i in range(1,8):  
#         client.write_registers(int(address), lc, unit = i)
#         time.sleep(.5)
#     for i in range(1,8):
#         client.write_registers(int(address), ld, unit = i)
#         time.sleep(.5)
#     for i in range(1,8):            
#         client.write_registers(int(address), le, unit = i)
#         time.sleep(.5)

data = client.read_input_registers(200,count=1, unit=1)
print(data)
temp = data.registers
print(temp)

while(int(tempTo*10) < int(temp[0])):
    data = client.read_input_registers(200,count=1, unit=1)
    temp = data.registers

#end time = time + 60 * num minutes
t_end = time.time() + 60 * 2

while time.time() < t_end:
    tmpData_Array = readTemp(tmpData_Array)
    df = df.append({'Time': time.time() , 'Temp1': tmpData_Array[0]}, ignore_index=True)
    time.sleep(1)


#turn every pannel off
#     for i in range(1,8):
client.write_registers(int(address), off, unit = 1)

Please input the temperature you would like to start the test at :300
ReadInputRegistersResponse (1)
[233]


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [96]:
temp

[233]

In [99]:
temp[0]

233

# Test Code for Optical Power Meter Preliminary testing
1. Bring every string of one color of LED to X% power
2. Every 2 inches take power measurments
3. Find the max power at that two inch line by moving the detector verticly
4. Log the max power
5. Continuosly log the temp every 5 seconds

In [1]:
################ Getting the coms with the optical power meter working ##############

import pyvisa
# from ThorlabsPM100 import ThorlabsPM100
rm = pyvisa.ResourceManager()
rm.list_resources()

('USB0::0x1313::0x8078::P0012289::INSTR',
 'ASRL4::INSTR',
 'ASRL5::INSTR',
 'ASRL6::INSTR',
 'ASRL7::INSTR',
 'ASRL8::INSTR')

In [3]:
rm = pyvisa.ResourceManager()
#opens the device and starts line of communication
OpticBoi = rm.open_resource('USB0::0x1313::0x8078::P0012289::INSTR')
#asks instriment its name
print(OpticBoi.query("*IDN?"))
#sets device to default state so state is known
OpticBoi.write("*RST")
#sets wavelength of optical power meter
OpticBoi.write('sense:corr:wav 850')

Thorlabs,PM100D,P0012289,2.4.0



20

In [5]:
################API for operatin the ThorLabs PM100D over PyVisa serial communication ##########################################
class OpticalPower:
    #function will initailize communication to device with given address
    #an error will be raised if communication is not established
    def __init__(self,address):
        rm = visa.ResourceManager()
        try:
            self.OpticBoi = rm.open_resource(address)
            #check we are connected to the correct device
            name = self.OpticBoi.query("*IDN?")
            #responce should be "Thorlabs,PM100D,P0012289,2.4.0"
            if ((str(name[0]) == 'T') & (name[1] == 'h')):
                print('Connected to OpticBoi')
            else:
                print('Device did not respond with correct name')
                logging.error('Device did not respond with correct name')
        except:
            print("Error connecting to device")
            logging.error("Error connecting to device")
            
   ##funcion writes the wavelength value given to optical power meter##
    def writeWav(self,wavelength):
        try:
            #writes voltage to power supply
            self.OpticBoi.write('sense:corr:wav ' + str(wavelength))
            #wait one second as to not send commands to quickly
            time.sleep(1)
            #querry for wavelenth set of power meter
            actualWav = self.OpticBoi.query('sense:corr:wav?')
            #if the wavelength read is equal to the wavelength sent no error raised
            if (float(wavelength) == float(actualWav)) :
                print('')
            else:
                print('actual wavelength does not match wavelegth writen')
                logging.error('actual wavelength does not match wavelegth writen')
        except Exception as e:
            print('wavelength write failed')
            logging.error('wavelength write failed')
            print(e)
            
    #function reads the power 
    def readPower(self):
        try:
            power = self.OpticBoi.query('measure:power?')
            return power
            
        except Exception as e:
            print('power read failed')
            logging.error('power read failed')
            print(e)
            
    #function will close device
    def close(self):
        try:
            self.OpticBoi.close()
        except Exception as e:
            print('faild to close ThorLabs PM100D')
            logging.error('faild to close ThorLabs PM100D')
            print(e)
            
    #function writes to device
    def write(self,com):
        try:
            self.OpticBoi.write(com)
        except Exception as e:
            print('failed to send command to ThorLabs PM100D')
            logging.error('failed to send command to ThorLabs PM100D')
            print(e)

# Code for test

In [7]:

def Opto_Test():
    
    #adress of register to write to
    #blue 1 is at 100
    #blue 2 is at 101
    #940 is at 102
    #850 is at 103
    
    address = 103 # starting with the 850's
    
    TestName = input("Please input the name of the test:") # set name of test
    
    dirLoc1 = "C:\\Users\\aaron\\Dropbox\\NewForge\\Summer2023\Europa_Simulator\OpticalPowerMeterTesting\OpticalData" + TestName + ".csv"
    dirLoc2 = "" + TestName + ".csv"
    
    log_format = "%(asctime)s::%(levelname)s::%(name)s::"\
             "%(filename)s::%(lineno)d::%(message)s"
    logging.basicConfig(filename='mylogs.log', level='ERROR', format=log_format)
    
    header1 = ['Time', 'Temp1', 'Temp2', 'Temp3', 'Temp4', 'Temp5', 'Temp6', 'Temp7']
    header2 = ['Time', 'Position', 'Intensity']

    
    client = mbc(method = "rtu", port="COM8",stopbits = 1, bytesize = 8, parity = 'N', baudrate= 115200, retries=0, timeout=.1)
    client.connect()
    
    
    
    
    df1 = pd.DataFrame(columns= header1)
    df2 = pd.DataFrame(columns= header2)
    
    
    
    
    
    df = df.append({'Time': time.time() , 'Temp1': tmpData_Array[0][0] , 'Temp2': tmpData_Array[1][0], 'Temp3': tmpData_Array[2][0] , 'Temp4': tmpData_Array[3][0], 'Temp5': tmpData_Array[4][0], 'Temp6': tmpData_Array[5][0] , 'Temp7': tmpData_Array[6][0]}, ignore_index=True)

    